# Titel 

## Abstract


## Requirements
`Python 3.14.2`

| Package       | Version   |
| ------------- | --------- |
| ImageIO       | 2.37.2    |
| matplotlib    | 3.10.8    |
| numpy         | 2.2.6     |
| opencv-python | 4.12.0.88 |
| scikit-image  | 0.26.0    |

```bash
pip install imageio==2.37.2 matplotlib==3.10.8 numpy==2.2.6 opencv-python==4.12.0.88 scikit-image==0.26.0
```

## Note
The following function is for visualisation purposes only and has no influence on the actual data processing. To use it throughout the notebook, the corresponding code block must be executed once.
After restarting the kernel or the project, all previously defined functions are lost. In this case, this code block must always be executed first before subsequent cells function correctly.

In [1]:
# source: https://stackoverflow.com/questions/3173320/text-progress-bar-in-terminal-with-block-characters
def progressBar(iterable, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd="\r"):
    def printProgressBar(iteration):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    
    printProgressBar(0)
    
    for i, item in enumerate(iterable):
        yield item
        printProgressBar(i + 1)
        
    # print new line on complete
    print()

## Preperation - Extraction of subframes
During the design sessions, the participants' interactions with CollabJam were recorded via screen capture. Since analysing all frames (in this specific case, approximately 57 minutes of playback time at 24 frames per second) would be too time-consuming and possibly pointless, subframes are extracted in the first step. 

The `interval` parameter can be used to control how many subframes per minute are to be saved. Our analysis uses 8 frames per minute here. The complete extraction may take several minutes.

In [2]:
import cv2
import os

# paths
video_path = r"Assets\screenrecording_designsession.mp4"
output_folder = r"Output\extracted_subframes"

# 1/x * 60 --> x Frames extracted per minute
interval = 1/8 * 60

# generate timestamp (MM:SS)
def format_time(seconds):
    mins = int(seconds) // 60
    secs = int(seconds) % 60
    return f"{mins:02d}:{secs:02d}"

# extract frames
def extract_screenshots(video_path, output_folder, interval):
    # gather metadata
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    screenshots = []

    num_shots = int(duration // interval)

    # extract frames
    for i in progressBar(range(num_shots + 1), total=num_shots + 1, prefix='Extracting frames:', suffix='Complete', length=40):
        timestamp = i * interval
        frame_number = int(timestamp * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()

        if not ret:
            continue

        # create timestamp text
        text = f"Zeit: {format_time(timestamp)} / {format_time(duration)}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        thickness = 2
        color = (255, 255, 255)
        outline = (0, 0, 0)
        pos = (30, 50)
        cv2.putText(frame, text, pos, font, font_scale, outline, thickness + 2, cv2.LINE_AA)
        cv2.putText(frame, text, pos, font, font_scale, color, thickness, cv2.LINE_AA)

        # save
        filename = os.path.join(output_folder, f"screenshot_{frame_number}.png")
        cv2.imwrite(filename, frame)
        screenshots.append(filename)

    cap.release()
    return screenshots

def main():
    # check if output_folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extract_screenshots(video_path, output_folder, interval)
    print(f"Done!")

if __name__ == "__main__":
    main()


Extracting frames: |████████████████████████████████████████| 100.0% Complete
Done!


## Processing and analysis of subframes

The analysis procedure comprised several consecutive steps. First, the previously extracted subframes were cropped using a mask to isolate only the relevant area of the user interface for analysis. 

### 1. Masking

The mask was created manually in an image editing programme and depends on the respective source material. White areas of the mask remain in the image after masking, while areas marked in black are removed or appear black in the masked image.
It is also important that the dimensions of the mask are identical to those of the extracted subframes.

<figure id="Fig.1_mask" style="--min-width: 15em">
    <figure>
        <img src="Assets\mask.png"/>
        <figcaption>mask</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\00_subframe.png"/>
        <figcaption>original subframe</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\01_masked.png"/>
        <figcaption>masked subframe</figcaption>
    </figure>
    <figcaption><strong>Figure 1: masking of the original subframe</strong></figcaption>
</figure>

### 2. Thresholding

In the next step, the cropped images were first converted to greyscale and then transformed into binary images.

<figure id="Fig.2_thresholding" style="--min-width: 15em">
    <figure>
        <img src="Assets\images_for_markdown\01_masked.png"/>
        <figcaption>result of the prevoius steps (masking)</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\02_thresholding.png"/>
        <figcaption>result of thresholding</figcaption>
    </figure>
    <figcaption><strong>Figure 2: thresholding</strong></figcaption>
</figure>

### 3. Morphological opening
Since this representation still contained unwanted image elements such as track markings, a morphological opening was applied to remove these disturbances and clean up the image structure.

<figure id="Fig.3_morphology" style="--min-width: 15em">
    <figure>
        <img src="Assets\images_for_markdown\02_thresholding.png"/>
        <figcaption>result of the prevoius steps (masking, thresholding)</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\03_morphology.png"/>
        <figcaption>after morphology</figcaption>
    </figure>
    <figcaption><strong>Figure 3: morphological opening</strong></figcaption>
</figure>

### 4. Contour detection
This was followed by the detection of the block contours.

<figure id="Fig.4_contours" style="--min-width: 15em">
    <figure>
        <img src="Assets\images_for_markdown\03_morphology.png"/>
        <figcaption>result of the prevoius steps (masking, thresholding, morphology)</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\04.1_contours_only.png"/>
        <figcaption>detected contours</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\04.2_contours_with_original.png"/>
        <figcaption>dectected contours with original subframe</figcaption>
    </figure>
    <figcaption><strong>Figure 4: contour detection</strong></figcaption>
</figure>

### 5. Lifetime map
Based on these contours, a so-called lifetime map was created, a data structure that assigns a lifetime value to each detected pixel from the contour detection. New or significantly changed block contours appear as brightly highlighted areas that fade over time. This logic creates a visual representation of the dynamics in the design process. The GIF animation created shows the development of the interactions over time by clearly highlighting changes in the timeline. In this way, the creative work process can be depicted intuitively and comprehensibly.

<figure id="Fig.5_lifetime_map" style="--min-width: 15em">
    <figure>
        <img src="Assets\images_for_markdown\04.1_contours_only.png"/>
        <figcaption>result of the prevoius steps (masking, thresholding, morphology, contour detection)</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\05_lifetime_map_after_first_frame.png"/>
        <figcaption>lifetime map after the first frame</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\05_lifetime_map_after_some_frames.png"/>
        <figcaption>lieftime map after some frames have been processed</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\06_overlay_progress.gif"/>
        <figcaption>final animation</figcaption>
    </figure>
    <figcaption><strong>Figure 5: lifetime map</strong></figcaption>
</figure>

### 6. Quantitative analysis
In addition to visual analysis, the design process was also evaluated quantitatively using the <a href="https://scikit-image.org/docs/0.25.x/auto_examples/transform/plot_ssim.html">Structural Similarity Index (SSIM)</a>. This value indicates how similar two images are in terms of their structure based on perception. In contrast to simple error measures (e.g. MSE), SSIM not only evaluates pixel-by-pixel differences, but also focuses on characteristics that are relevant to human vision. 

The SSIM ultimately calculates a dimensionless scalar value that indicates how structurally similar two images are. The value typically ranges between 0 and 1, with 1 representing nearly identical images and smaller values indicating increasing structural differences.
In this use case, however, it is not the similarity but the visual change between successive frames that is of interest. Therefore, the inverse of the SSIM (1 − SSIM) is used. High values mark points in time with significant changes, while low values indicate phases with little or no activity. This allows changes over time to be clearly and intuitively identified in the diagram. In this context, the inverse is referred to as the **SSIM activity** or **visual activity**.

#### 6.1 SSIM activity diagram
The calculation was performed for all consecutive frames of the generated animation and the results were visualised in a diagram that shows the dynamics of change over the entire period.

<figure id="Fig.5_lifetime_map" style="--min-width: 15em">
    <img src="Assets\images_for_markdown\activity_diagram_ssim.svg"/>    
    <figcaption><strong>Figure 6: SSIM-activity diagram</strong></figcaption>
</figure>

#### 6.2 Excerpt from the list of highest SSIM activity values
In addition, a sorted list was generated that lists the frame pairs with the highest values, including the corresponding frame numbers and file names, enabling targeted tracing of the most significant moments of change. 

| Platz | SSIM-activity  | Frame A | File A                | Frame B | File B                |
|-------|----------------|---------|-----------------------|---------|-----------------------|
| 1     | 0.007          | 350     | screenshot_64440.png  | 351     | screenshot_64620.png  |
| 2     | 0.007          | 13      | screenshot_3420.png   | 14      | screenshot_3600.png   |
| 3     | 0.005          | 349     | screenshot_64260.png  | 350     | screenshot_64440.png  |
| 4     | 0.004          | 357     | screenshot_65700.png  | 358     | screenshot_66240.png  |
| 5     | 0.004          | 185     | screenshot_34560.png  | 186     | screenshot_34740.png  |
| 6     | 0.003          | 40      | screenshot_8460.png   | 41      | screenshot_8640.png   |
| 7     | 0.003          | 171     | screenshot_32040.png  | 172     | screenshot_32220.png  |
| 8     | 0.003          | 348     | screenshot_64080.png  | 349     | screenshot_64260.png  |
| 9     | 0.003          | 396     | screenshot_73260.png  | 397     | screenshot_73440.png  |
| 10    | 0.003          | 41      | screenshot_8640.png   | 42      | screenshot_8820.png   |

### 7. Cleaning up the subframes
Since the authoring tool used (<a href="https://github.com/TactileVision/CollabJam-Client">CollabJam V1.1.0</a>) employs popover menus that appear centrally in the image, there were isolated cases of unwanted fluctuations in the SSIM values and visual disturbances in the generated GIF animation. These popover elements led to abrupt changes in the image, which were incorrectly interpreted by the analysis procedure as design activity.

<figure id="Fig.7_lifetime_map" style="--min-width: 15em">
    <figure>
        <img src="Assets\images_for_markdown\cleanup\ok_frame.png"/>
        <figcaption>ok subframe</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\cleanup\ok_lifetime_map.png"/>
        <figcaption>ok lifetime map</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\cleanup\bad_frame.png"/>
        <figcaption>poor subframe</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\cleanup\bad_lifetime_map.png"/>
        <figcaption>poor lifetime map</figcaption>
    </figure>
    <figure>
        <img src="Assets\images_for_markdown\cleanup\bad_animation.gif"/>
        <figcaption>poor animation</figcaption>
    </figure>
    <figcaption><strong>Figure 7: Effects of poor subframes using the example of frames 241 and 242, which have the highest SSIM activity value in the generated list. </strong></figcaption>
</figure>

In order not to distort the significance of the evaluation, all subframes with exceptionally high SSIM activity values were checked manually (e.g. by using the list of SSIM activity values and the corresponding subframes. Frames in which the change could be attributed to the display of user interfaces or other interface elements were removed from the analysis (deleted from `Output\extracted_subframes`). This ensured that only actual changes in the design work process were taken into account.

For the screenrecording of the designsessions, this process was already done. Cleaned up subframes are located at `Assets\cleaned_subframes`. To run the analysis on these frames, either switch up the subframepath or replace all extracted subframes at `Output\extracted_subframes` by the cleaned up subframes.

List of removed frames:
- screenshot_1620.png
- screenshot_1800.png
- screenshot_2520.png
- screenshot_2700.png
- screenshot_2880.png
- screenshot_3060.png
- screenshot_6660.png
- screenshot_43560.png
- screenshot_65880.png
- screenshot_66060.png
- screenshot_72360.png

<style>
    figure:has(> figure) {
    --column-gap: 1em;

    display: flex;
    flex-wrap: wrap;
    justify-content: center;
    align-items: baseline;
    column-gap: var(--column-gap);
    }

    figure > figure {
        min-width: min(var(--min-width), 100%);
        flex-basis: 0;
        flex-grow: 1;
    }

    figcaption {
        width: 100%;
        text-align: center;
    }
</style>

In [4]:
import cv2
import numpy as np
import os
import imageio.v2 as iio
import re
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

# paths
subframes = r"Output\extracted_subframes"
mask_path = r"Assets\mask.png"
output_folder = r"Output"

# settings for diagramm
labelSize = 16

def generate_particle_overlay_frames_generator(image_paths, mask, max_lifetime, debug=False):
    if not image_paths:
        return

    if debug and not os.path.exists(output_folder + "\\debug_frames"):
        os.makedirs(output_folder + "\\debug_frames")

    first_img = cv2.imread(image_paths[0])
    if first_img is None:
        raise ValueError(f"Could not load image: {image_paths[0]}")

    h, w, _ = first_img.shape

    lifetime_map = np.zeros((h, w), dtype=np.int32)
    kernel = np.ones((3, 3), np.uint8)


    for i, image_path in enumerate(image_paths):
        img_original = cv2.imread(image_path)
        if img_original is None:
            print(f"Could not load image and skipping it: {image_path}")
            continue
        
        # grayscaling
        img_gray = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
        img_masked = cv2.bitwise_and(img_gray, img_gray, mask=mask)
        
        # thresholding
        _, thresh = cv2.threshold(img_masked, 180, 255, cv2.THRESH_BINARY_INV)
        
        # morphological opening
        processed_thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        
        final_binary_image = cv2.bitwise_and(processed_thresh, processed_thresh, mask=mask)
        
        # find contours
        contours, _ = cv2.findContours(final_binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # update lifetime map
        lifetime_map[lifetime_map > 0] -= 1

        current_frame_blocks_mask = np.zeros((h, w), dtype=np.uint8)
        for cnt in contours:
            cv2.drawContours(current_frame_blocks_mask, [cnt], -1, 255, -1)

        lifetime_map[current_frame_blocks_mask == 255] = max_lifetime
        
        # all intermediate results can be saved for debugging purposes
        if debug:
            frame_dir = os.path.join(output_folder + "\\debug_frames", f"frame_{i:04d}")
            if not os.path.exists(frame_dir):
                os.makedirs(frame_dir)
            
            # safe intermediate steps
            cv2.imwrite(os.path.join(frame_dir, "01_masked.png"), img_masked)
            cv2.imwrite(os.path.join(frame_dir, "02_thresholding.png"), thresh)
            cv2.imwrite(os.path.join(frame_dir, "03_morphology.png"), processed_thresh)
            
            # contours

            # detected contours only
            contours_img_black_bg = np.zeros_like(img_original)
            cv2.drawContours(contours_img_black_bg, contours, -1, (0, 255, 0), 2)
            cv2.imwrite(os.path.join(frame_dir, "04.1_contours_only.png"), contours_img_black_bg)

            # detected contours overlapping with original subframe
            contours_img = img_original.copy()
            cv2.drawContours(contours_img, contours, -1, (0, 255, 0), 2)
            cv2.imwrite(os.path.join(frame_dir, "04.2_contours_with_original.png"), contours_img)
            
            # lifetime map
            lifetime_map_visual = np.clip((lifetime_map.astype(np.float32) / max_lifetime) * 255, 0, 255).astype(np.uint8)
            cv2.imwrite(os.path.join(frame_dir, "05_lifetime_map.png"), lifetime_map_visual)
        
        # generate frame for animation

        # normalization of lifetime
        opacity_intensity_channel = (lifetime_map.astype(np.float32) / max_lifetime)

        # generate black image
        overlay_visual = np.zeros((h, w, 3), dtype=np.float32)

        # pass opacity values to green channel only
        overlay_visual[:, :, 1] = opacity_intensity_channel

        # gamma / alpha correction
        # make gradients appear more ‘natural’ and ‘less digital’
        # particles fade out more gradually rather than linearly
        alpha_channel_3d = opacity_intensity_channel[..., np.newaxis]
        blended_float = overlay_visual * alpha_channel_3d
        
        # convert to final frame
        blended_float = np.clip(blended_float, 0, 1) 
        blended_frame_uint8 = (blended_float * 255).astype(np.uint8)

        # deliver frame of animation
        yield blended_frame_uint8

def calculate_metrics(prev_frame, current_frame):    
    # visual activity measure based on structural dissimilarity.
    # 0.0 = no change, higher values = stronger visual change  
    # [:, :, 1] -> use only the green channel for faster computation, as red and blue are always zero
    ssim_score = ssim(prev_frame[:, :, 1], current_frame[:, :, 1], channel_axis=-1, data_range=255)
    return  1.0 - ssim_score

def plot_diagram(values, type_name, ylabel, filename, color):
    plt.style.use("ggplot")
    plt.figure(figsize=(15, 6))
    plt.plot(range(1, len(values) + 1), values, color=color, marker='o', linestyle='-', markersize=4)
    plt.xlabel("Frame", fontsize=labelSize)
    plt.ylabel(ylabel, fontsize=labelSize)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, filename))
    plt.close()
    print(f"{type_name}-Diagram saved.")

def save_sorted_list(differences, top_n):
    differences.sort(key=lambda x: x['activity'], reverse=True)
    output_lines = ["--- Frame pairs sorted by descending difference ---\n"]
    for j, diff in enumerate(differences[:top_n]):
        activity, (f1, f2), (n1, n2) = diff['activity'], diff['pair'], diff['frame_numbers']
        output_lines.append(f"{j+1}. Place: visual activity = {activity:.3f}\n   - Frame {n1}: {os.path.basename(f1)}\n   - Frame {n2}: {os.path.basename(f2)}\n")
    
    with open(os.path.join(output_folder, "sorted_differences.txt"), "w", encoding="utf-8") as f:
        f.write("\n".join(output_lines))
    print("Saved sorted list.")

def main():
    image_paths = sorted([
        os.path.join(subframes, f)
        for f in os.listdir(subframes)
        if f.lower().endswith(('.png', '.jpg'))
    ])

    # ensure that frames are processed in the correct order
    screenshot_number_regex = re.compile(r'^screenshot_(\d+)\.png$', re.IGNORECASE)
    temp_files = []
    for f in os.listdir(subframes):
        if f.lower().endswith(('.png', '.jpg', '.jpeg')):
            match = screenshot_number_regex.search(f)
            num = int(match.group(1)) if match else -1
            temp_files.append((num, os.path.join(subframes, f)))
    temp_files.sort(key=lambda x: x[0])
    image_paths = [path for num, path in temp_files if num != -1]

    total_frames = len(image_paths)
    if total_frames == 0: raise ValueError("No frames found. Please first generate subframes based on video material (see section Preparation - Extraction of subframes).")
    
    mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask_img is None: raise ValueError("Could not load mask.")

    # container
    activity_values = []
    differences_list = []
    
    print(f"starting analysis for {total_frames} Frames...")

    # init generator and gif-writer
    # if debug == True, all intermediate steps (masking, thresholding, etc.) for every frame are saved at ./Output/debug_frames
    debug = False

    # defines the number of frames required for a particle to disappear completely
    # high value (e.g. 100) --> particles leave long, trail-like traces. This creates a strong ‘ghosting effect’. This is helpful for visualising movement paths over a longer period of time.
    # low value (e.g. 5) --> particles disappear almost immediately. The image appears more turbulent, but shows the current state of the respective frame more precisely.
    max_lifetime = 20

    # setup generator
    frame_gen = generate_particle_overlay_frames_generator(image_paths, mask_img, max_lifetime=max_lifetime, debug=debug)
    gif_path = os.path.join(output_folder, "overlay_progress.gif")    
    prev_frame = None
    
    # duration determines how long a single image (frame) is displayed before the next one appears
    # This value can be adjusted depending on the number of frames. 
    # If the animation is too fast, increase the value (e.g. to 100 for 10 FPS). 
    # If it is too slow, decrease it (e.g. to 33 for approx. 30 FPS).
    # FPS = 1000 / duration
    duration = 50

    with iio.get_writer(gif_path, mode='I', duration=duration, loop=0) as writer:
        for i, current_frame in enumerate(progressBar(frame_gen, total=total_frames, prefix='Processing:', suffix='Complete', length=50)):
            
            # 1. calculate all metrics
            if prev_frame is not None:
                activity = calculate_metrics(prev_frame, current_frame)
                activity_values.append(activity)
                
                # safe data for list
                differences_list.append({
                    'activity' : activity,
                    'pair': (image_paths[i-1], image_paths[i]),
                    'frame_numbers': (i-1, i)
                })

            # 2. prepare animation frame
            gif_frame = current_frame.copy()            
            writer.append_data(gif_frame)
            prev_frame = current_frame.copy()

    # SSIM Diagramm
    plot_diagram(activity_values, "SSIM-based activity", "visual activity (1 − SSIM)", "activity_diagram_ssim.svg", "firebrick")
    # safe list
    save_sorted_list(differences_list, top_n=40)
    print("Done!")

if __name__ == "__main__":
    main()

starting analysis for 410 Frames...
Processing: |██████████████████████████████████████████████████| 100.0% Complete
SSIM-based activity-Diagram saved.
Saved sorted list.
Done!
